# ResNet - Model

### Importação dos pacotes

In [2]:
from keras.applications.resnet import ResNet152
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

### Organizando a rede neural convolucional

In [3]:
# realizando a transferência de aprendizagem com a rede ResNet
base_model = ResNet152(weights='imagenet', include_top=False)

# pegando a última camada da DenseNet
x = base_model.output

# adicionando uma camada de Global Average Pooling
x = GlobalAveragePooling2D()(x)

# adicionando a camada de saída da rede 
predictions = Dense(units = 2, activation = 'softmax')(x)

# concatenando a rede importada com a rede definida (ajuste fino)
model = Model(inputs=base_model.input, outputs=predictions)

234700800/234698864 [==============================] - 5s 0us/step


In [4]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [5]:
# compilando os parâmetros da rede
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001),
              loss = 'categorical_crossentropy', metrics = ['acc'])

In [8]:
# definindo as flags iniciais  
model.trainable = True
set_trainable = False

# para a arquitetura DenseNet, a rede será retreinada a partir da camada 'conv5_block1_1_conv'
for layer in model.layers:
    if layer.name == 'conv5_block1_1_conv':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 4242 // 32, 
                              validation_data = valid_generator, validation_steps = 1061 // 32,
                              callbacks = callbacks, epochs = 10)